# Tipos de Encoding - Codificação

* Tabela ASCCII não contém caracteres especiais contruida para o idioma inglês.

ISO 8859-1 ou mais mais conhecido como latin-1

ISO/IEC 8859-1 é uma codificação de caracteres do alfabeto latino, a primeira parte da ISO 8859. Foi desenvolvida pela ISO, e posteriormente passou a ser mantida pela ISO e pela IEC. Este padrão é a base de outros mapeamentos amplamente usados como o ISO-8859-1 e o Windows-1252

* https://docs.python.org/3/library/codecs.html#standard-encodings

* https://upload.wikimedia.org/wikipedia/commons/d/dd/ASCII-Table.svg

# Usando for para leitura de arquivo

* Usar o for consome menos memória que o método readlines ou readline.

# Um pouco mais sobre o with

Utilizamos o comando with para gerenciar o contexto de utilização do arquivo. Além de arquivos, podemos utilizar o with para gerenciar processos que precisam de uma pré e pós condição de execução; por exemplo: abrir e fechar o arquivo, realizar conexão com o banco de dados, sockets, entre outros.

O objeto que está sendo manipulado pelo with precisa implementar dois métodos mágicos: __enter__() e __exit__().

O método __enter__() é executado logo no início da chamada da função e retorna uma representação do objeto que está sendo executada no contexto (ou context guard). Ao final, o método __exit__() é invocado, e o contexto da execução, finalizado.

Para saber um pouco mais sobre o statement, leia este post do blog preshing.com (em inglês).

* https://preshing.com/20110920/the-python-with-statement-by-example/